## get the data 

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

Soil_types_directory= 'C:/Users/user/Google Drive/TCD20/python/dataset/Soil types'
data_image=pd.read_csv(os.path.join(Soil_types_directory,'all_image_64.csv'))
data_image_mean=pd.read_csv(os.path.join(Soil_types_directory,'all_image_mean.csv'))
data_image_Selecting=pd.read_csv(os.path.join(Soil_types_directory,'Selecting_image_64.csv'))

In [194]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import  confusion_matrix,multilabel_confusion_matrix
from sklearn.metrics import classification_report,precision_recall_fscore_support

def def_GridSearchCV(pipeline_object,param_grid,X,Y):
    # run GridSearchCV
    grid_search = GridSearchCV(pipeline_object,param_grid , scoring='accuracy', cv=5, n_jobs=-1, verbose=2)
    grid_search.fit(X,Y) 
    #cols =['param_'+param for param in list(grid_search.param_grid[0].keys())] +['mean_test_score','std_test_score','rank_test_score']
    cv_results_df = pd.DataFrame(grid_search.cv_results_).sort_values(by = 'mean_test_score', ascending=False)#[cols]
    filter_col = [col for col in cv_results_df if col.startswith('param')]+['mean_test_score','std_test_score']
    cv_results_df= cv_results_df[filter_col]
    best_params=grid_search.best_params_
    best_score=grid_search.best_score_
    param_grid_cv=grid_search.param_grid
    return    cv_results_df, param_grid_cv,best_params,best_score, grid_search

def  Summary_results_CVׁׂ(cv_results_df):
     cv_results_df['str_model']=cv_results_df['param_classifier'].apply(str)
     base_pramter=cv_results_df.groupby(by=['str_model']).apply(lambda df: df.query('mean_test_score==mean_test_score.max()'))
     filter_col = [col for col in cv_results_df if col.startswith('param')]+['mean_test_score','std_test_score']
     base_pramter=  base_pramter[filter_col].reset_index(drop=True).sort_values(by = 'mean_test_score', ascending=False)#
     return   base_pramter 
     
 

def base_model_predict(grid_search,X,Y):
    # predict train
    predict_train= grid_search.predict(X)
    predict_df=Y.copy().assign( predict_class=predict_train )
    
    
    classification_report1=classification_report(predict_df['classes'],
                                                      predict_df['predict_class'],
                                                      labels=grid_search.classes_,
                                                      output_dict=True)
  
    multi_confusion_matrix1=multilabel_confusion_matrix(predict_df['classes'],
                                       predict_df['predict_class'],
                                       labels=grid_search.classes_)
    
      
    confusion_matrix1= confusion_matrix(predict_df['classes'],
                                       predict_df['predict_class'],
                                       labels=grid_search.classes_)
    
    return  predict_df,classification_report1,  confusion_matrix1,multi_confusion_matrix1
 
    
def base_model_summary(grid_search,classification_report,multi_confusion_matrix,confusion_matrix):
    
    # classification_report to DataFrame
    required_fields=['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil','macro avg', 'weighted avg']#, 'weighted avg'
    classification_report_dict={k:v['precision'] for k,v in classification_report.items() if k in required_fields}
    classification_report_dict.update({'accuracy':classification_report1['accuracy']})
    classification_report_df=pd.DataFrame(classification_report_dict,index=['precision']).T
    classification_report_df= classification_report_df.sort_values(by = 'precision', ascending=False)#
    
    # confusion_matrix to  DataFrame
    dict_matrix=dict()
    classes=grid_search.classes_
    for i,l in zip(range(multi_confusion_matrix.shape[0]),classes):
        cm=multi_confusion_matrix1[i,:,:]
        df_cm = pd.DataFrame(cm,index= ['other',l], columns=['other',l])
        #confusion_matrix, index=class_names, columns=class_names,
        dict_matrix.update({l:df_cm})

    confusion_matrix_df= pd.DataFrame(confusion_matrix,index=classes, columns=classes)
    return classification_report_df, dict_matrix, confusion_matrix_df
    

## examine the best model and parameters
LogisticRegression,KNeighbors,SVC, RandomForest







## param_grid

In [173]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


#from sklearn.metrics import accuracy_score
#from sklearn.metrics import  confusion_matrix,multilabel_confusion_matrix
#from sklearn.metrics import classification_report,precision_recall_fscore_support

full_pipeline = Pipeline(steps=[
   ('preprocessor',None),
    ('classifier', None)
])

param_grid= [
    {
   
     'classifier': [ LogisticRegression(solver='liblinear')],
     'classifier__penalty': ['l1', 'l2'],
     'classifier__C': [0.1,0.5,1,5,10],
     },
   {  
        'classifier': [KNeighborsClassifier()],
        'classifier__n_neighbors': [2,3,7,11,15,21,31],
    },
     {
       
        'classifier': [SVC()],
        'classifier__C': [0.1,0.5,1,5,10,20,50],
         'classifier__gamma':[0.00001,0.00005,0.0001,0.0005,0.001,0.005,0.01,0.05]
    },
    {
         
        'classifier': [RandomForestClassifier()],
        'classifier__n_estimators': [50,100,200,500],
         'classifier__max_depth':[1, 10, 50, 100],
         'classifier__min_samples_split':[2,5,10,15]
    }
]

In [174]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures,QuantileTransformer,PowerTransformer
from sklearn.preprocessing import  MinMaxScaler,StandardScaler,RobustScaler

#from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn import set_config


full_pipeline1 = Pipeline(steps=[
  
   ('preprocessor',None),
   ('norm',MinMaxScaler()), 
   ('classifier', SVC(C=10,gamma=0.0005))
   ])

full_pipeline2 = Pipeline(steps=[
    
   ('preprocessor',None),
   ( 'norm',MinMaxScaler()),
   ('classifier',RandomForestClassifier(n_estimators=500,max_depth=50,min_samples_split=10))
   ])


full_pipeline3 = Pipeline(steps=[
   ('preprocessor',None),
   ( 'norm',MinMaxScaler()),
   ('classifier',KNeighborsClassifier(n_neighbors=11))
   ])



param_grid_processes= [
    {},
    
    {
     'preprocessor':[PolynomialFeatures()],   
     'preprocessor__degree':[2,3,4] 
      },
    {'preprocessor':[QuantileTransformer()]
    },
    {'preprocessor':[PowerTransformer()],
   'preprocessor__method':['yeo-johnson']   
}]
    
  

#  data image all

###  train_test_split data

In [ ]:
data_image2=data_image.groupby(by=['classes']).sample(frac=0.005, random_state=42)
print(data_image2.shape)

x_data=data_image2[['r','g','b']]#,'image_classes'
y_data=data_image2[['classes']]
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

### examine necessary processes


In [ ]:
cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(full_pipeline3 ,param_grid_processes,X_train,y_train)
print(best_params,best_score)
cv_results_df

###  rnn the models

In [195]:
cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(full_pipeline ,param_grid,X_train,y_train)
print(best_params,best_score)
base_pramter=Summary_results_CVׁׂ(cv_results_df)

predict_df,classification_report1,confusion_matrix1,multi_confusion_matrix1=base_model_predict(grid_search,X_train, y_train)



classification_report_df, dict_matrix, confusion_matrix_df= base_model_summary(grid_search,classification_report1,multi_confusion_matrix1,confusion_matrix1)

Fitting 5 folds for each of 137 candidates, totalling 685 fits


C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'classifier': SVC(C=10, gamma=0.0005), 'classifier__C': 10, 'classifier__gamma': 0.0005} 0.7272015655577299


## predict and  Summary base_model 

In [196]:
classification_report_df
#dict_matrix['Black Soil']
#confusion_matrix_df

,precision
Yellow Soil,0.927473
Laterite Soil,0.853933
macro avg,0.759768
weighted avg,0.757835
accuracy,0.757730
Black Soil,0.707143
Cinder Soil,0.690860
Peat Soil,0.619433


# data_image_Selecting

In [ ]:

data_image_Selecting2=data_image_Selecting.groupby(by=['classes']).sample(frac=0.05, random_state=42)
print(data_image_Selecting2.shape)
from sklearn.model_selection import train_test_split

x_data=data_image_Selecting2[['r','g','b']]#,'image_classes'
y_data=data_image_Selecting2[['classes']]
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)



cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(full_pipeline ,param_grid,X_train,y_train)
print(best_params,best_score)
base_pramter=Summary_results_CVׁׂ(cv_results_df)
#predict_df,classification_report1,confusion_matrix1,multi_confusion_matrix1=base_model_predict(grid_search,X_train, y_train)
base_pramter

# data_image_mean
 

In [ ]:
x_data=data_image_mean[['r','g','b']]#,'image_classes'
y_data=data_image_mean[['classes']]
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(full_pipeline ,param_grid,X_train,y_train)
print(best_params,best_score)
base_pramter=Summary_results_CVׁׂ(cv_results_df)
#predict_df,classification_report1,confusion_matrix1,multi_confusion_matrix1=base_model_predict(grid_search,X_train, y_train)
base_pramter